# Classificació de cel·lules usant la U-Net

**TODO: Descripcio**

*La següent cel·la s'ha d'executar sempre*

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive/Universidad/Alumno colaborador/u_net_folder

/content/gdrive/My Drive/Universidad/Alumno colaborador/u_net_folder


In [ ]:
!git clone https://github.com/bmalcover/u_cells
%cd u_cells
!git checkout 'dev/iker'
%cd ..

Cloning into 'u_cells'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 8), reused 14 (delta 5), pack-reused 0
Unpacking objects: 100% (22/22), done.
/content/u_cells
Branch 'dev/iker' set up to track remote branch 'dev/iker' from 'origin'.
Switched to a new branch 'dev/iker'
/content


**DATA**

Només executar si dins la carpeta *u_net_folder* no hi ha la carpeta *unet_color_quatre*

In [ ]:
#!wget -O dataset.zip https://www.dropbox.com/s/3zfz89hik6989h5/dataset.zip?dl=1
!wget -O dataset_four.zip https://www.dropbox.com/s/0v6rdf3xhoge0vh/unet_color_quatre.zip?dl=1 
#!wget -O dataset_gt.zip https://www.dropbox.com/s/wvjlpqykck6jwc6/unet_color_tres.zip?dl=1
#!wget -O dataset_gros.zip https://www.dropbox.com/s/e7ndtiqce74gqli/sub.zip?dl=1
 
#!unzip dataset.zip
!unzip dataset_four.zip > /dev/null
#!unzip dataset_gt.zip
#!unzip dataset_gros.zip

--2021-03-09 11:20:21--  https://www.dropbox.com/s/0v6rdf3xhoge0vh/unet_color_quatre.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/0v6rdf3xhoge0vh/unet_color_quatre.zip [following]
--2021-03-09 11:20:21--  https://www.dropbox.com/s/dl/0v6rdf3xhoge0vh/unet_color_quatre.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9339d6e4518217b131f48fac96.dl.dropboxusercontent.com/cd/0/get/BKWnYLPeIMYPd0XNgtPumkGm-yNIBJ5N_iHyl7JsUYFty6zn5Y_FPffKAN6bWl_HNXRKxEQr1S02cL31EtfAJ8SOhLL29wtGIJg-CwrHrctl9uWi-PBQRFSGunTLMwbTcF5imsTqa1bhJYYURxQ95XKv/file?dl=1# [following]
--2021-03-09 11:20:22--  https://uc9339d6e4518217b131f48fac96.dl.dropboxusercontent.com/cd/0/get/BKWnYLPeIMYPd0XNgtPumkGm-yNIBJ5N_iHyl7JsUYFty6zn5Y_FPffKAN6bW

Llibreries

In [3]:
# %tensorflow_version 1.x
import io
from datetime import datetime

from matplotlib import pyplot as plt

from numpy.random import seed
import tensorflow as tf
from tensorflow.keras import metrics
import keras
import keras.backend as K
from PIL import Image


import numpy as np
import cv2

seed(1)
#print(tf.__version__)

Funcionalitats propies



In [4]:
%cd u_cells/

from model import *
from data import *

%cd ..

/content/gdrive/My Drive/Universidad/Alumno colaborador/u_net_folder/u_cells
/content/gdrive/My Drive/Universidad/Alumno colaborador/u_net_folder


### Preparació de les dades i generació de la xarxa
---

In [5]:
test_path_root = "./unet_color_quatre/test/image/"
test_path = test_path_root + "*.png"
train_path = './unet_color_quatre/train'
generator_path = None #'./generator5/'
tb_dir = "./log/" + datetime.now().strftime("%Y/%m/%d-%H:%M:%S")
# tb_cb = keras.callbacks.TensorBoard(tb_dir, write_graph=True, update_freq=10)

data_gen_args = dict(rotation_range=0.2,
						width_shift_range=0.05,
						height_shift_range=0.05,
						shear_range=0.05,
						zoom_range=0.05,
						horizontal_flip=True,
						fill_mode='nearest')

train_generator = trainGenerator(2, train_path,'image','label',data_gen_args,save_to_dir = generator_path, mask_color_mode = "rgb", decode_flag=True, target_size=(512,512)) #AQui
val_generator = trainGenerator(2, './unet_color_quatre/test','image','label',data_gen_args,save_to_dir = generator_path, mask_color_mode = "rgb", decode_flag=True, target_size=(512,512)) #AQui

model = get_small_unet(n_filters=64, bn = True, input_size=(512,512,1), output_channels = 1, loss_func = dice_coef_loss) ### AqUI!!!!!
# model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
#tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 512, 512, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 1)  4           leaky_re_lu[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         batch_normalization[0][0]        
______________________________________________________________________________________________

### Entrenament
---

In [6]:
model.fit(train_generator, validation_data = val_generator, validation_steps=100, steps_per_epoch=300, epochs=10) # 300, 5

Found 33 images belonging to 1 classes.
Found 33 images belonging to 1 classes.
Epoch 1/10
  6/300 [..............................] - ETA: 8:14 - loss: 0.4068 - categorical_accuracy: 0.8235

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-47f11947a269>", line 1, in <module>
    model.fit(train_generator, validation_data = val_generator, validation_steps=100, steps_per_epoch=300, epochs=10) # 300, 5
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 855, in _call
    return self._stateless_fn(*args, **kwds)  # pylint: disable=not-callable
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/function.py", line 2943, in __call__
    filtered_flat_ar

KeyboardInterrupt: ignored

### Resultats - avaluació
---


In [ ]:
testGene = testGenerator(test_path, target_size=(512, 512))
results = model.predict(testGene, 16)

In [ ]:
plt.rcParams["figure.figsize"] = (30,15)

In [ ]:
def remove_border_cells(contours, shape):
    """
    Removes the objects from the borders of the image.

    A border of an image is the zone near the start or the end of the matrix. 
    The index of this points are near 0 and near the shape of the image. The 
    contours checked as a parameter don't has that exactly index so is needed 
    to has an acceptable error.

    Args:
        contours: List of numpy arrays, every array its a different contour. The array has two
                  columns and many rows as points in the contour. Depending of the appoximation
                  method used
        shape:

    Returns:

    """
    center_contours = []

    for cont in contours:
        cont = np.squeeze(cont)
        border = not np.all(
            (cont[:, 0] > 15) & (cont[:, 1] > 15) & (cont[:, 0] < shape[1] - 15) &
            (cont[:, 1] < shape[0] - 15))

        if not border:
            center_contours.append(np.array([cont]).reshape((cont.shape[0], 1, 
                                                             cont.shape[1])))

    return center_contours

def get_iou(ground, prediction, th, debug=False, remove_border_segs = False):
  assert ground.shape[2] == prediction.shape[2]
  
  ground = ground.astype(np.float32) / ground.max()
  ious = [0] * ground.shape[2]
  
  if debug:
    fig = plt.figure(1,(16, 12))
    idx = 1

  for channel_idx in range(0, ground.shape[2]):

    channel_gt = ground[:,:, channel_idx]
    channel_pred = np.copy(prediction[:, :, channel_idx])
    
    channel_pred = cv2.resize(channel_pred, (channel_gt.shape[1], channel_gt.shape[0]), interpolation = cv2.INTER_NEAREST) 

    channel_pred[channel_pred <= th] = 0
    channel_pred[channel_pred > th] = 1

    if remove_border_segs:
      contours, _ = cv2.findContours((channel_pred * 255).astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
      contours = remove_border_cells(contours, channel_pred.shape)

      channel_pred = np.zeros_like(channel_pred)
      channel_pred = cv2.drawContours(channel_pred, contours, -1, 1, -1)
 
    intersection = cv2.bitwise_and(channel_gt, channel_pred)
    intersection_area = np.count_nonzero(intersection)
    
    union = cv2.bitwise_or(channel_gt, channel_pred)
    union_area = np.count_nonzero(union)
    
    if debug:
      l = [channel_gt, channel_pred, intersection, union, (union-intersection)]
      titles = ["GT", "PRED", "INTERSEC", "UNION", "DIFF"]
    
      for i in range(len(l)):
        plt.title(titles[(i-1) % 5])
        plt.subplot(3,5, idx)
        plt.imshow(l[i], cmap="gray");
        idx += 1
    
    ious[channel_idx] += round(intersection_area / union_area, 3)
    
  return ious

In [ ]:
alpha = 0.5

ious = []
for idx, res in enumerate(results):
  gt_image = cv2.imread("./unet_color_quatre/test/label/" + str(idx) + ".png")
  # cv2 reads the images in BGR we convert them into rgb
  gt_image = cv2.cvtColor(gt_image, cv2.COLOR_BGR2RGB) 

  gt_image = decode(gt_image)

  iou = get_iou(gt_image, res, alpha)
  ious.append(iou)

  if idx < 10:
    print("Image 0"+ str(idx) + ": " + str(iou))
  else:
    print("Image "+ str(idx) + ": " + str(iou))

print("###########################################")
print("Mean: ", np.mean(ious, axis = 0))

La següent cel·la només serveix per evaluar el funcionament de _get_iou_

In [ ]:
index = 3

ground = cv2.imread("./unet_color_quatre/test/label/" + str(index) + ".png")
prediction = results[index]
th = 0.5
ground = decode(ground)
get_iou(ground, prediction, th, True)

In [ ]:
#BRUTOR
index = 3
ground = cv2.imread("./unet_color_quatre/test/label/" + str(index) + ".png")
ground = decode(ground)

ground = cv2.resize(ground, (512, 512), interpolation = cv2.INTER_NEAREST) 
prediction = results[index]

g = K.constant(ground)
p = K.constant(prediction)
        # y_true, y_pred

plt.subplot(1,2, 1)
plt.imshow(ground, cmap="gray");
plt.subplot(1,2, 2)
plt.imshow(prediction, cmap="gray");

dice_coef_loss(g, p)


Visualització dels resultats en format *RAW*

In [ ]:
res = results[6] # Canviar aquest index per veure diferents imatges

fig = plt.figure(1,(16,12))
dim = res.shape[2]
for i in range(dim):
  plt.subplot(1, dim, i+1)
  plt.imshow(res[:,:,i], cmap="gray");

Guardar resultats en una carpeta anomenada Out

*Nota: Es sobreescriuen els resultats anteriors*

In [ ]:
import os

!rm -rf out/
!mkdir out

for idx, r in enumerate(results):
  for c_idx in range(0, r.shape[2]):
    path = "./out/" + str(c_idx) + "/"
    if not os.path.exists(path): 
      os.makedirs(path)

    cv2.imwrite(path + str(idx) + ".png", r[:,:,c_idx]*255)

In [ ]:
from google.colab import files

!zip -r ./out.zip ./out > /dev/null
files.download("./out.zip");